In [6]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, preprocessing
import tensorflow_datasets as tfds

In [7]:
max_length = 200
n_words = 10000
dim_embedding  = 256
epochs = 20
batch_size = 256

In [11]:
def load_data():
    (X_train,y_train),(X_test, y_test) = datasets.imdb.load_data(num_words=n_words)
    X_train = preprocessing.sequence.pad_sequences(X_train,maxlen=max_length)
    X_test = preprocessing.sequence.pad_sequences(X_test,maxlen=max_length)
    return (X_train, y_train), (X_test, y_test)

In [13]:
def build_model():
    model  = models.Sequential()
    model.add(layers.Embedding(n_words,dim_embedding,input_length=max_length))
    model.add(layers.Dropout(0.3))
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1,activation='sigmoid'))
    return model

In [14]:
(X_train, y_train), (X_test, y_test) = load_data()
model = build_model()

17465344/17464789 [==============================] - 3s 0us/step


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 256)          2560000   
_________________________________________________________________
dropout (Dropout)            (None, 200, 256)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,593,025
Trainable params: 2,593,025
Non-trainable params: 0
______________________________________________

In [27]:
import os
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%load_ext tensorboard
%tensorboard --logdir {logs_base_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 11000), started 5:35:24 ago. (Use '!kill 11000' to kill it.)

In [28]:
import datetime
logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [29]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
score = model.fit(X_train,y_train,batch_size = batch_size,
                  epochs=epochs,callbacks=[tensorboard_callback],validation_data=(X_test,y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 25s 1ms/sample - loss: 0.1495 - accuracy: 0.9482 - val_loss: 0.3359 - val_accuracy: 0.8711
Epoch 2/20
25000/25000 [==============================] - 22s 894us/sample - loss: 0.1284 - accuracy: 0.9556 - val_loss: 0.3601 - val_accuracy: 0.8671
Epoch 3/20
25000/25000 [==============================] - 23s 907us/sample - loss: 0.1121 - accuracy: 0.9636 - val_loss: 0.3874 - val_accuracy: 0.8629
Epoch 4/20
25000/25000 [==============================] - 22s 893us/sample - loss: 0.1005 - accuracy: 0.9683 - val_loss: 0.4204 - val_accuracy: 0.8573
Epoch 5/20
25000/25000 [==============================] - 22s 886us/sample - loss: 0.0888 - accuracy: 0.9749 - val_loss: 0.4542 - val_accuracy: 0.8561
Epoch 6/20
25000/25000 [==============================] - 22s 900us/sample - loss: 0.0774 - accuracy: 0.9787 - val_loss: 0.4863 - val_accuracy: 0.8522
Epoch 7/20
25000/25000 [======================

In [ ]:
score = model.evaluate(X_test,y_test,batch_size=batch_size,callbacks=[tensorboard_callback])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])